<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 4:** Running State Chains</font>

<br>

In the previous notebook, we introduced some key LangChain Expression Language (LCEL) material regarding runnables. By now, you should be comfortable with both internal and external reasoning, as well as how to develop pipelines that facilitate it! In this notebook, we will make our way towards more advanced paradigms that will allow us to orchestrate more complex dialog management strategies and begin to execute on longer-form document reasoning.

<br>

### **Learning Objectives:**

- Learning how to leverage runnables to orchestrate interesting LLM systems.  
- Understanding how running state chains can be used for dialog management and iterative decision-making.

<br>

### **Questions To Think About:**

- Would there ever be a use for a single-module variant of the running state chain that is not constantly querying the environment for input?
- You may notice that the JSON prediction is actually working pretty well. It might not always work so well depending on the questions and the JSON format complexity. What kinds of issues do you expect to encounter in this regard?
- What kinds of approaches can you think of completely swapping prompts as part of the running state chain?

<br>

### **Notebook Source:**

- This notebook is part of a larger [**NVIDIA Deep Learning Institute**](https://www.nvidia.com/en-us/training/) course titled [**Building RAG Agents with LLMs**](https://learn.next.courses.nvidia.com/courses/course-v1:DLI+S-FX-15+V1/about). If sharing this material, please give credit and link back to the original course.

<br>


### **Environment Setup:**

In [ ]:
## Necessary for Colab, not necessary for course environment
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio

## If you're in colab and encounter a typing-extensions issue,
##  restart your runtime and try again
# from langchain_nvidia_ai_endpoints._common import NVEModel

In [ ]:
from getpass import getpass
import requests
import os

hard_reset = False  ## <-- Set to True if you want to reset your NVIDIA_API_KEY
while "nvapi-" not in os.environ.get("NVIDIA_API_KEY", "") or hard_reset:
    try: 
        assert not hard_reset
        response = requests.get("http://docker_router:8070/get_key").json()
        assert response.get('nvapi_key')
    except: response = {'nvapi_key' : getpass("NVIDIA API Key: ")}
    os.environ["NVIDIA_API_KEY"] = response.get("nvapi_key")
    try: requests.post("http://docker_router:8070/set_key/", json={'nvapi_key' : os.environ["NVIDIA_API_KEY"]}).json()
    except: pass
    hard_reset = False
    if "nvapi-" not in os.environ.get("NVIDIA_API_KEY", ""):
        print("[!] API key assignment failed. Make sure it starts with `nvapi-` as generated from the model pages.")

print(f"Retrieved NVIDIA_API_KEY beginning with \"{os.environ.get('NVIDIA_API_KEY')[:9]}...\"")
from langchain_nvidia_ai_endpoints._common import NVEModel
NVEModel().available_models

----

<br>

## **Part 1:** Keeping Variables Flowing

In the previous examples, we were able to implement interesting logic in our standalone chains by **creating**, **mutating**, and **consuming** states. These states were passed around as dictionaries with descriptive keys and useful values, and the values would be used to supply follow-up routines with the info they need to operate!

**Recall the zero-shot classification example from the last notebook:**

In [ ]:
%%time
## ^^ This notebook is timed, which will print out how long it all took

from langchain_core.runnables import RunnableLambda
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from typing import List, Union
from operator import itemgetter

## Zero-shot classification prompt and chain
zsc_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "One word answer. Output the most relevant topic option. [Options : {options}]"
    )),
    ("user", "{input} = ")
])

## Useful method for mistral, which is currently tuned to output numbered outputs
def EnumParser(*idxs):
    '''Method that pulls out values from a mistral model that outputs numbered entries'''
    idxs = idxs or [slice(0, None, 1)]
    entry_parser = lambda v: v if (' ' not in v) else v[v.index(' '):]
    out_lambda = lambda x: [entry_parser(v).strip() for v in x.split("\n")]
    return StrOutputParser() | RunnableLambda(lambda x: itemgetter(*idxs)(out_lambda(x)))

## Define your simple instruct_model
instruct_llm = ChatNVIDIA(model="mistral_7b") | EnumParser(0)  ## TODO: Try out some more models

zsc_chain = zsc_prompt | instruct_llm

def zsc_call(input, options=["car", "boat", "airplane", "bike"]):
    return zsc_chain.invoke({"input" : input, "options" : options})

print("-" * 80)
print(zsc_call("Should I take the next exit, or keep going to the next one?"))

print("-" * 80)
print(zsc_call("I get seasick, so I think I'll pass on the trip"))

print("-" * 80)
print(zsc_call("I'm scared of heights, so flying probably isn't for me"))

<br>

This chain makes several design decisions that make it very easy to use, key among them the following:

**We want it to act like a function, so all we want it to do is generate the output and return it.**

This makes the chain extremely natural for inclusion as a module in a larger chain system. For example, the following chain will take a string, extract the most likely topic, and then generate a new sentence based on the topic:



In [ ]:
%%time
## ^^ This notebook is timed, which will print out how long it all took

gen_prompt = ChatPromptTemplate.from_template(
    "Make a new sentence about the the following topic: {topic}. Be creative!"
)

gen_chain = gen_prompt | instruct_llm

input_msg = "I get seasick, so I think I'll pass on the trip"
options = ["car", "boat", "airplane", "bike"]

({'topic' : zsc_chain} | gen_chain).invoke({"input" : input_msg, "options" : options})

<br>

However, it's a bit problematic when you want to keep the information flowing since we lose the topic and input variables in generating our response. If we wanted do something with both the output and the input, we'd need a to make sure that both variables pass through.

Lucky for us, we can use the mapping runnable (i.e. interpretted from a dictionary or using manual `RunnableMap`) to pass both of the variables through by assigning the output of our chain to just a single key and letting the other keys propagate as desired. Alternatively, we could also use `RunnableAssign` to merge the state-consuming chain's output with the input dictionary by default.

With this technique, we can propagate whatever we want through our chain system:

In [ ]:
%%time
## ^^ This notebook is timed, which will print out how long it all took

from langchain.schema.runnable import RunnableBranch, RunnablePassthrough
from langchain.schema.runnable.passthrough import RunnableAssign
from functools import partial

def RPrint(preface=""):
    def print_and_return(x, preface=""):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

big_chain = (
    RPrint("State: ")
    ## Manual mapping. Can be useful sometimes and inside branch chains
    | {'input' : lambda d: d.get('input'), 'topic' : zsc_chain}
    | RPrint("State: ")
    ## RunnableAssign passing. Better for running state chains by default
    | RunnableAssign({'generation' : gen_chain})
    | RPrint("State: ")
    ## Using the input and generation together
    | RunnableAssign({'combination' : (
        ChatPromptTemplate.from_template(
            "Consider the following passages:"
            "\nP1: {input}"
            "\nP2: {generation}"
            "\n\nCombine the ideas from both sentences into one simple one."
        )
        | instruct_llm
    )})
)

big_chain.invoke({
    "input" : "I get seasick, so I think I'll pass on the trip",
    "options" : ["car", "boat", "airplane", "bike", "unknown"]
})


----

<br>

## **Part 2:** Running State Chain

The example above is just a toy example and, if anything, showcases the drawbacks of chaining many LLM calls together for internal under-the-hood reasoning. However, the ability to keep information flowing through a chain is invaluable for making complex chains that can accumulate useful state information or operate in a multi-pass capacity.

Specifically, a very simple but effective chain is a **Running State Chain** which enforces the following properties:
- A **"running state"** is a dictionary that contains all of the variables that the system cares about.
- A **"branch"** is a chain that can pull in the running state and can degenerate it into a response.
- A **branch** can only be ran inside a **RunnableAssign** scope, and the branchs' inputs should come from the **running state**.

> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/running_state_chain.png" width=1000px/>
<!-- > <img src="https://drive.google.com/uc?export=view&id=1Oo7AauYGj4dxepNReRG2JezmvQLyqXsN" width=1000px/> -->

You can think of the running state chain abstraction as a functional variant of a Pythonic class with state variables (or attributes) and functions (or methods).
- The chain is like the abstract class that wraps all of the functionality.
- The running state are like the attributes (which should always be accessible).
- The branches are like the class methods (which can pick and choose which attributes to use).
- The `.invoke` or similar process is like the `__call__` method that runs through the branches in order.

**By forcing this paradigm in your chains:**
- You can keep state variables propagating through your chain, allowing your internals to access whatever is necessary and accumulating state values for use later.
- You can also pass the outputs of your chain back through as your inputs, allowing a "while-loop"-style chain that keeps updating and building on your running state.

The rest of this notebook will include two exercises that flesh out the running state chain abstraction for two additional use-cases: **Knowledge Bases** and **Database-Querying Chatbots**.

----

<br>

## **Part 3:** Implementing a Knowledge Base with Running State Chain

After understanding the basic structure and principles of a Running State Chain, we can explore how this approach can be extended to manage more complex tasks, particularly in creating dynamic systems that evolve through interaction. This section will focus on implementing a **knowledge base** accumulated using **json-enabled slot filling**:

- **Knowledge Base:** A store of information that's relevant for our LLM to keep track of.
- **JSON-Enabled Slot Filling:** The technique of asking an instruction-tuned model to output a json-style format (which can include a dictionary) with a selection of slots, relying on the LLM to fill these slots with useful and relevant information.

<br>

#### **Defining Our Knowledge Base**

To build a responsive and intelligent system, we need a method that not only processes inputs but also retains and updates essential information through the flow of conversation. This is where the combination of LangChain and Pydantic becomes pivotal. [**Pydantic**](https://docs.pydantic.dev/latest/), a popular Python validation library, is instrumental in structuring and validating data models. As one of its features, Pydantic offers structured "model" classes that validate objects (data, classes, themselves, etc.) with simplified syntax and deep rabbitholes of customization options. This framework is used throughout LangChain and comes up as a necessary component for use cases that involve data coersion.

One thing that a "model" is very good for is defining a class with expected arguments and some special ways to validate them! In this course, we won't focus too much on the validation scripts, but those interested can start by checking out the [**Pydantic Validator guide**](https://docs.pydantic.dev/1.10/usage/validators/) (though the topics do get pretty deep pretty fast). For our purposes, we can construct a `BaseModel` class and define some `Field` variables to create a structured **Knowledge Base** like so:

In [ ]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import Dict, Union, Optional

class KnowledgeBase(BaseModel):
    ## Fields of the BaseModel, which will be validated/assigned when the knowledge base is constructed
    topic: str = Field('general', description="Current conversation topic")
    user_preferences: Dict[str, Union[str, int]] = Field({}, description="User preferences and choices")
    session_notes: str = Field("", description="Notes on the ongoing session")
    unresolved_queries: list = Field([], description="Unresolved user queries")
    action_items: list = Field([], description="Actionable items identified during the conversation")

print(repr(KnowledgeBase(topic = "Travel")))

<br>

The true strength of this approach lies in the additional LLM-centric functionalities provided by LangChain which we can integrate for our use-cases. One such feature is the `PydanticOutputParser` which enhances the Pydantic objects with capabilities like automatic format instruction generation.

In [ ]:
from langchain.output_parsers import PydanticOutputParser

instruct_string = PydanticOutputParser(pydantic_object=KnowledgeBase).get_format_instructions()
print(instruct_string)

<br>

This functionality generates instructions for creating valid inputs to the Knowledge Base, which in turn helps the LLM by providing a concrete one-shot example of the desired output format.

<br>

#### **Runnable Extraction Module**

Knowing that we have this Pydantic object which can be used to generate good LLM instructions, we can make a Runnable that wraps the functionality of our Pydantic class and streamlines the prompting, generating, and updating of the knowledge base:

In [ ]:
################################################################################
## Definition of RExtract
def RExtract(pydantic_class, llm, prompt):
    '''
    Runnable Extraction module
    Returns a knowledge dictionary populated by slot-filling extraction
    '''
    parser = PydanticOutputParser(pydantic_object=pydantic_class)
    instruct_merge = RunnableAssign({'format_instructions' : lambda x: parser.get_format_instructions()})
    def preparse(string):
        if '{' not in string: string = '{' + string
        if '}' not in string: string = string + '}'
        string = (string
            .replace("\\_", "_")
            .replace("\n", " ")
            .replace("\]", "]")
            .replace("\[", "[")
        )
        # print(string)  ## Good for diagnostics
        return string
    return instruct_merge | prompt | llm | preparse | parser

################################################################################
## Practical Use of RExtract
instruct_model_big = ChatNVIDIA(model = "mixtral_8x7b") | StrOutputParser()

parser_prompt = ChatPromptTemplate.from_messages([
    ("system", "Update the knowledge base: {format_instructions}. Only use information from the input."),
    ("user", "{input}"),
])

extractor = RExtract(KnowledgeBase, instruct_model_big, parser_prompt)

knowledge = extractor.invoke({'input' : "I love flowers so much! The orchids are amazing! Can you buy me some?"})
knowledge

<br>

Do keep in mind that this process can fail due to the fuzzy nature of LLM prediction, especially with models that are not optimized for instruction-following! For this process, it's important to have a strong instruction-following LLM with extra checks and graceful failure routines. 

<br>

#### **Dynamic Knowledge Base Updates**

Finally, we can create a system that continually updates the Knowledge Base throughout the conversation. This is done by feeding the current state of the Knowledge Base, along with new user inputs, back into the system for ongoing updates.

The following is an example system that shows off both the formulation's power of filling details as well as the limitations of assuming that filling performance will be as good as general response performance:

In [ ]:
class KnowledgeBase(BaseModel):
    firstname: str = Field('unknown', description="Chatting user's first name, unknown if unknown")
    lastname: str = Field('unknown', description="Chatting user's last name, unknown if unknown")
    location: str = Field('unknown', description="Where the user is located")
    hints: str = Field('unknown', description="Hints to help answer other questions")
    response: str = Field('unknown', description="Ideal response based on last user input")


parser_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "The user just responsed. Please update the knowledge base based on the response."
        " This information will be acted on to respond to the user in the next interaction."
        " Do not hallucinate any details, and make sure the knowledge base is not redundant."
        " Update the entries frequently to adapt to the conversation flow."
        "\n{format_instructions}"
    )), ("user", "CURRENT KNOWLEDGE BASE: {know_base}\nUser: {input}"),
])

instruct_model_big = ChatNVIDIA(model = "mixtral_8x7b") | StrOutputParser()

extractor = RExtract(KnowledgeBase, instruct_model_big, parser_prompt)
info_update = RunnableAssign({'know_base' : extractor})

state = {'know_base' : KnowledgeBase()}
state['input'] = "My name is Carmen Sandiego! Guess where I am?"
state = info_update.invoke(state)
print(state)

state['input'] = "The United States is a big place! Can you be more specific?"
state = info_update.invoke(state)
print(state)

state['input'] = "Yeah, I'm in New Orleans... How did you know?"
state = info_update.invoke(state)
print(state)

<br>

This example demonstrates how a running state chain can be effectively utilized to manage a conversation with evolving context and requirements, making it a powerful tool for developing sophisticated interactive systems.

The next sections of this notebook will expand on these concepts, exploring two specific applications: Document Knowledge Bases and Database-Querying Chatbots. These exercises will further elucidate the versatility and effectiveness of the Running State Chain abstraction in various real-world scenarios.

----

<br>

## **Part 4: [Exercise]** Airline Customer Service Bot

In this exercise, we can expand on the tools we've learned about to implement a simple but effective dialog manager chatbot. For this exercise, we will make an airline support bot that wants to help a client find out about their flight!

Let's create a simple database-like interface to get some customer information from a dictionary!

In [ ]:
#######################################################################################
## Function that can be queried for information. Implementation details not important
def get_flight_info(d: dict) -> str:
    """
    Example of a retrieval function which takes a dictionary as key. Resembles SQL DB Query
    """
    req_keys = ['first_name', 'last_name', 'confirmation']
    assert all((key in d) for key in req_keys), f"Expected dictionary with keys {req_keys}, got {d}"

    ## Static dataset. get_key and get_val can be used to work with it, and db is your variable
    keys = req_keys + ["departure", "destination", "departure_time", "arrival_time", "flight_day"]
    values = [
        ["Jane", "Doe", 12345, "San Jose", "New Orleans", "12:30 PM", "9:30 PM", "tomorrow"],
        ["John", "Smith", 54321, "New York", "Los Angeles", "8:00 AM", "11:00 AM", "Sunday"],
        ["Alice", "Johnson", 98765, "Chicago", "Miami", "7:00 PM", "11:00 PM", "next week"],
        ["Bob", "Brown", 56789, "Dallas", "Seattle", "1:00 PM", "4:00 PM", "yesterday"],
    ]
    get_key = lambda d: "|".join([d['first_name'], d['last_name'], str(d['confirmation'])])
    get_val = lambda l: {k:v for k,v in zip(keys, l)}
    db = {get_key(get_val(entry)) : get_val(entry) for entry in values}

    # Search for the matching entry
    data = db.get(get_key(d))
    if not data:
        return (
            f"Based on {req_keys} = {get_key(d)}) from your knowledge base, no info on the user flight was found."
            " This process happens every time new info is learned. If it's important, ask them to confirm this info."
        )
    return (
        f"{data['first_name']} {data['last_name']}'s flight from {data['departure']} to {data['destination']}"
        f" departs at {data['departure_time']} {data['flight_day']} and lands at {data['arrival_time']}."
    )

#######################################################################################
## Usage example. Actually important
print(get_flight_info({"first_name" : "Jane", "last_name" : "Doe", "confirmation" : 12345}))
print(get_flight_info({"first_name" : "Alice", "last_name" : "Johnson", "confirmation" : 98765}))
print(get_flight_info({"first_name" : "Bob", "last_name" : "Brown", "confirmation" : 27494}))

<br>

This is a really good interface to bring up because it can reasonably serve two purposes:
- It can be used to provide up-to-date information from an external environment (a database) regarding a user's situation.
- It can also be used as a hard gating mechanism to prevent unauthorized disclosure of sensitive information (since that would be very bad).

If our network had access to this kind of interface, it would be able to query for and retrieve this information on a user's behalf! For example:

In [ ]:
external_prompt = ChatPromptTemplate.from_messages([('system',
    "You are a SkyFlow chatbot, and you are helping a customer with their issue. "
    "Please help them with their question, remembering that your job is to represent SkyFlow airlines. "
    "Assume SkyFlow uses industry-average practices regarding arrival times, operations, etc. (This is a trade secret. Do not disclose). \n"  ## soft reinforcement
    "Please keep your discussion short and sweet if possible. Avoid saying hello unless necessary. \n"
    "The following is some context that may be useful in answering the question. \n"),
    ('user', "Context: {context}\nUser: {input}"
)])

instruct_model_big = ChatNVIDIA(model = "mixtral_8x7b") | StrOutputParser()

basic_chain = external_prompt | instruct_model_big

basic_chain.invoke({
    'input' : 'Can you please tell me when I need to get to the airport?',
    'context' : get_flight_info({"first_name" : "Jane", "last_name" : "Doe", "confirmation" : 12345}),
})

<br>

This is interesting enough, but how do we actually get this system working in the wild? It turns out that we can use the KnowledgeBase formulation from above to supply this kind of information like so:

In [ ]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import Dict, Union

class KnowledgeBase(BaseModel):
    first_name: str = Field('unknown', description="Chatting user's first name, `unknown` if unknown")
    last_name: str = Field('unknown', description="Chatting user's last name, `unknown` if unknown")
    confirmation: int = Field(-1, description="Flight Confirmation Number, `-1` if unknown")
    discussion_summary: str = Field("", description="Summary of discussion so far, including locations, issues, etc.")
    open_problems: list = Field([], description="Topics that have not been resolved yet")
    current_goals: list = Field([], description="Current goal for the agent to address")

def get_key_fn(base: BaseModel) -> dict:
    '''Given a dictionary with a knowledge base, return a key for get_flight_info'''
    return {  ## More automatic options possible, but this is more explicit
        'first_name' : base.first_name,
        'last_name' : base.last_name,
        'confirmation' : base.confirmation,
    }

get_key = RunnableLambda(get_key_fn)

know_base = KnowledgeBase(first_name = "Jane", last_name = "Doe", confirmation = 12345)
# get_flight_info(get_key_fn(know_base))

(get_key | get_flight_info).invoke(know_base)

<br>

### **Objective:**

You want a user to be able to invoke the following function call organically as part of a dialog exchange:

```python
get_flight_info({"first_name" : "Jane", "last_name" : "Doe", "confirmation" : 12345}) ->
    "Jane Doe's flight from San Jose to New Orleans departs at 12:30 PM tomorrow and lands at 9:30 PM."
```

`RExtract` is provided such that the following knowledge base syntax can be used:
```python
known_info = KnowledgeBase()
extractor = RExtract(KnowledgeBase, InstructLLM(), parser_prompt)
results = extractor.invoke({'info_base' : known_info, 'input' : 'My message'})
known_info = results['info_base']
```

**Design a chatbot that implements the following features:**
- The bot should start off by making small-talk, possibly helping the user with non-sensitive queries which don't require any private info access.
- When the user starts to ask about things that are database-walled (both practically and legally), tell the user that they need to provide the relevant information.
- When the retrieval is successful, the agent will be able to talk about the database-walled information.

**This can be done with a variety of techniques, including the following:**
- **Prompt Engineering and Context Parsing**, where the overall chat prompt stays roughly the same but the context is manipulated to to change agent behavior. For example, a failed db retrieval could be changed into an injection of natural-language instructions for how to resolve the problem such as *`"Information could not be retrieved with keys {...}. Please ask the user for clarification or help them with known information."`*
- **"Prompt Passing,"** where the active prompts are passed around as state variables and can be overridden by monitoring chains.
- **Branching chains** such as [**`RunnableBranch`**](https://python.langchain.com/docs/expression_language/how_to/routing) or more custom solutions that implement an conditional routing mechanism.
    - In the case of [`RunnableBranch`](https://python.langchain.com/docs/expression_language/how_to/routing), a `switch` syntax of the style:
        ```python
        from langchain.schema.runnable import RunnableBranch
        RunnableBranch(
            ((lambda x: 1 in x), RPrint("Has 1 (didn't check 2): ")),
            ((lambda x: 2 in x), RPrint("Has 2 (not 1 though): ")),
            RPrint("Has neither 1 not 2: ")
        ).invoke([2, 1, 3]);  ## -> Has 1 (didn't check 2): [2, 1, 3]
        ```

Some prompts and a gradio loop are provided that might help with the effort, but the agent will currently just hallucinate! Please implement the internal chain to try and retrieve the relevant information. Before trying to implement, look over the default behavior of the model and note how it might hallucinate or forget things.

In [ ]:
from langchain.schema.runnable import (
    RunnableBranch,
    RunnableLambda,
    RunnableMap,       ## Wrap an implicit "dictionary" runnable
    RunnablePassthrough,
)
from langchain.schema.runnable.passthrough import RunnableAssign

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, SystemMessage, ChatMessage, AIMessage
from typing import Iterable
import gradio as gr

external_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are a SkyFlow chatbot, and you are helping a customer with their issue. Please chat with them! Stay concise and clear!"
        " Your running knowledge base is: {know_base}. This is for you only; Do not mention it! \nUsing that, we retrieved the following: {context}\n"
        " If they provide info and the retrieval fails, ask to confirm their first/last name and confirmation. Do not ask them any other personal info."
        " If it's not important to know about their flight, do not ask. The checking happens automatically; you cannot check manually."
    )),
    ("assistant", "{output}"),
    ("user", "{input}"),
])

##########################################################################
## Knowledge Base Things

class KnowledgeBase(BaseModel):
    first_name: str = Field('unknown', description="Chatting user's first name, `unknown` if unknown")
    last_name: str = Field('unknown', description="Chatting user's last name, `unknown` if unknown")
    confirmation: Optional[int] = Field(None, description="Flight Confirmation Number, `-1` if unknown")
    discussion_summary: str = Field("", description="Summary of discussion so far, including locations, issues, etc.")
    open_problems: str = Field("", description="Topics that have not been resolved yet")
    current_goals: str = Field("", description="Current goal for the agent to address")

parser_prompt = ChatPromptTemplate.from_messages([
    ('system', (
        "You are a chat assistant representing the airline SkyFlow, and are trying to track info about the conversation. "
        "You have just recieved a message from the user. Please fill in the schema based on the chat. \n{format_instructions}"
    )), ('user', "{know_base} is the current state. New exchange: You said {output} and they replied {input}")
])

fail_str = (
    "You cannot access user's flight/account details until they provide "
    "first name, last name, and flight confirmation code."
)

## Your goal is to invoke the following through natural conversation
# get_flight_info({"first_name" : "Jane", "last_name" : "Doe", "confirmation" : 12345}) ->
#     "Jane Doe's flight from San Jose to New Orleans departs at 12:30 PM tomorrow and lands at 9:30 PM."

## TODO: Pick a working model that you're happy with
instruct_llm = (ChatNVIDIA(model="mixtral_8x7b") | StrOutputParser())
chat_model = (ChatNVIDIA(model="mixtral_8x7b") | StrOutputParser())

## The external chain is already supported, and just streams the output
external_chain = external_prompt | chat_model

## The internal chain is invoked in the loop, but is not defined yet

#####################################################################################
## START TODO: Define the extractor and internal chain to satisfy the objective

extractor = RunnableAssign({'know_base' : (lambda x: None)})
internal_chain = RunnableAssign({'context' : (lambda x: "No context")})

## END TODO
#####################################################################################

state = {'know_base' : KnowledgeBase()}

def chat_gen(message, history=[], return_buffer=False):

    ## Pulling in, updating, and printing the state
    global state
    state['input'] = message
    state['history'] = history
    state['output'] = "" if not history else history[-1][1]

    ## Generating the new state from the internal chain
    state = internal_chain.invoke(state)
    print("State after chain run:", state)

    ## Streaming the results
    buffer = ""
    for token in external_chain.stream(state):
        buffer += token
        yield buffer

chatbot = gr.Chatbot(value = [[None, "Hello! I'm your SkyFlow agent! How can I help you?"]])
demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue()

try:
    ## NOTE: This should also give you a temporary public link which can be
    ## used to access this info on the public web while the session is live.
    demo.launch(debug=True, share=True, show_api=False)
    demo.close()
except Exception as e:
    demo.close()
    print(e)
    raise e

<br>

----

<br>

**NOTE:**
- You may need to explicitly hit the STOP button and try to relaunch your gradio interface if it hangs up after an exception. This is a known Jupyter Notebook environment issue which should not be experienced in dedicated Gradio-running files.
- **Your chat directive is duplicated here for quick access:**
```python
## Your goal is to invoke the following through natural conversation
get_flight_info({
    "first_name" : "Jane",
    "last_name" : "Doe",
    "confirmation" : 12345,
}) -> "Jane Doe's flight from San Jose to New Orleans departs at 12:30 PM tomorrow and lands at 9:30 PM."
```
- **To confirm that your system works, you could try the following dialog or something similar:**
```
> How's it going?
> Can you tell me a bit about skyflow?
> Can you tell me about my flight?
> My name is Jane Doe and my flight confirmation is 12345
> Can you tell me when I should get to my flight?
```
- **Solutions To Exercises Can Be Found In The Solutions Directory.** This is the first exercise with a noted solution, and additional exercises from the future notebooks will be found there.

-----

<br>

## **Part 5:** Wrap-Up

The goal of this notebook was to introduce some more advanced LangChain material revolving around the use of knowledge bases and running state chains! The exercise here was pretty involved, so congrats on finishing it!

### <font color="#76b900">**Great Job!**</font>

### **Next Steps:**
1. **[Optional]** Revisit the **"Questions To Think About" Section** at the top of the notebook and think about some possible answers.
2. Continue on to the next video, which will talk about **Document Loading and Summarization**.
3. After the video, go on to the corresponding notebook on **Document Loading and Summarization**.

---

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>